In [22]:
import numpy as np  
import pandas as pd
from pandas import Series, DataFrame
import tensorflow as tf
from sklearn.model_selection import train_test_split

from tensorflow.python.ops import confusion_matrix
from tensorflow.python.ops import math_ops

import matplotlib.pyplot as plt

In [23]:
#Data=pd.read_csv('C:/Users/dkoch/Documents/tensorflow/R_Input.csv')
Data=pd.read_csv('R_Input.csv')
Data=Data.fillna(0)
Data['dprio']=Data['dprio']/(10*np.std(Data['dprio']))

Data['dprio0']=Data['dprio']**0
Data['dprio2']=Data['dprio']**2
Data['dprio3']=Data['dprio']**3
Data=Data.rename(columns={'dprio': 'dprio1'})
Data['pax_total']=Data['pax_total']-Data['group_pax']
Data['pax_total']=Data['pax_total'].apply(lambda x: x if x>0 else 0)

direction=pd.get_dummies(Data['direction'],drop_first=True)
direction.columns=['direction:'+x for x in direction.columns]

whichholiday_left_LUX=pd.get_dummies(Data['whichholiday_left_LUX'],drop_first=True)
whichholiday_left_LUX.columns=['whichholiday_left_LUX:'+x for x in whichholiday_left_LUX.columns]

whichholiday_right_LUX=pd.get_dummies(Data['whichholiday_right_LUX'],drop_first=True)
whichholiday_right_LUX.columns=['whichholiday_right_LUX:'+x for x in whichholiday_right_LUX.columns]

Data=pd.concat([Data,direction,whichholiday_left_LUX,whichholiday_right_LUX],axis=1)

Data.index=Data['calendar_date']+'_'+Data['direction']

Data['weekday']=Data['calendar_date'].apply(lambda x: pd.to_datetime(x).weekday()+1)
Data['yearweek']=Data['calendar_date'].apply(lambda x: pd.to_datetime(x).isocalendar()[1])

In [24]:
dprio=Data[['dprio'+str(i) for i in range(0,4)]].drop_duplicates().transpose()
dprio.columns=range(365,-1,-1)
dprio=dprio[[x for x  in range(0,366)]]

X_data=Data.drop_duplicates(subset=['calendar_date','direction'])[[x for x in Data.columns if 'direction:' in x]\
+['weekday','yearweek']\
+[x for x in Data.columns if 'whichholiday_left_LUX:' in x]\
+[x for x in Data.columns if 'whichholiday_right_LUX:' in x]\
+['holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']]

In [25]:
X_data=X_data[['direction:O','weekday','yearweek',
'holiday_from_left_LUX','holiday_from_right_LUX','holiday_to_left_LUX','holiday_to_right_LUX']]

Data['concat']=Data['calendar_date']+'_'+Data['direction']
y_data=Data[['concat','dprio1','pax_total']].pivot(index='concat',columns='dprio1',values='pax_total')

In [26]:
X_train=X_data.loc[[x for x in X_data.index if '2019' not in x],:]
X_test=X_data.loc[[x for x in X_data.index if '2019' in x],:]
y_train=y_data.loc[[x for x in y_data.index if '2019' not in x],:]
y_test=y_data.loc[[x for x in y_data.index if '2019' in x],:]

n_input=X_train.shape[1]
n_hidden_1=n_input*2
n_hidden_2=4
n_output=y_train.shape[1]

In [27]:
#dprio.loc['dprio1',:]=(dprio.loc['dprio1',:]-np.mean(dprio.loc['dprio1',:]))/np.std(dprio.loc['dprio1',:])
#dprio.loc['dprio1',:]=-dprio.loc['dprio1',:]

In [28]:
#X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.3, random_state=1)

X_train=X_train.values
X_test=X_test.values
y_train=y_train.values
y_test=y_test.values

# dprio.loc['dprio0',:]=dprio.loc['dprio0',:]/365
# dprio.loc['dprio2',:]=(dprio.loc['dprio1',:]/365)**2
# dprio.loc['dprio3',:]=(dprio.loc['dprio1',:]/365)**3
# dprio.loc['dprio1',:]=dprio.loc['dprio1',:]/365
dprio=dprio.values

In [29]:
# Parameters
learning_rate = 10
training_epochs = 100

X = tf.placeholder("float", [None, n_input])
Z = tf.placeholder("float", [4, 366])
y = tf.placeholder("float", [None, n_output])

keep_prob = tf.placeholder(tf.float32)

# # Dictionary of Weights and Biases
# weights = {
#   'h1': tf.Variable(tf.zeros([n_input, n_hidden_1])),
#   'h2': tf.Variable(tf.zeros([n_hidden_1, n_hidden_2])),
# }

# biases = {
#   'b1': tf.Variable(tf.zeros([n_hidden_1])),
#   'b2': tf.Variable(tf.zeros([n_hidden_2])),
# }

# # Dictionary of Weights and Biases
weights = {
   'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1],mean=0,stddev=1)),
   'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2],mean=0,stddev=1)),
   #'out': tf.Variable(tf.random_normal([n_hidden_2, n_output]))
   #'out': tf.Variable(dprio.reshape(4,366),dtype=tf.float32)  
}

biases = {
   'b1': tf.Variable(tf.random_normal([n_hidden_1],mean=0,stddev=1)),
   'b2': tf.Variable(tf.random_normal([n_hidden_2],mean=0,stddev=1)),
   #'out': tf.Variable(tf.random_normal([n_output]))
}

# Model Forwar§d Propagation step
def forward_propagation(x,z):
    # Hidden layer1
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.sigmoid(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    #layer_2 = tf.nn.softmax(layer_2)

    # Output fully connected layer
    #out_layer=tf.add(tf.matmul(layer_2, weights['out']),biases['out'])
    out_layer = tf.matmul(layer_2, z)
    out_layer = tf.exp(out_layer)
    return out_layer


# Model Outputs
yhat = forward_propagation(X,Z)

In [30]:
cost = tf.reduce_mean(tf.square(yhat-y))
# our mean squared error cost function
#train = tf.train.GradientDescentOptimizer(0.001).minimize(cost)

#cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=yhat))
#cost=tf.losses.mean_squared_error(y, yhat)
train = tf.train.GradientDescentOptimizer(0.1).minimize(cost)

In [31]:
saver = tf.train.Saver()

# launch a session to run our graph defined above. 
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        # get a sample of the dataset and run the optimizer, which calculates a forward pass and then runs the backpropagation algorithm to improve the weights
        train.run(feed_dict = {X: X_train, y: y_train, Z: dprio.reshape(4,366)})
        # every 100 iterations, print out the accuracy
        # accuracy and loss are both functions that take (x, y) pairs as input, and run a forward pass through the network to obtain a prediction, and then compares the prediction with the actual y.
        loss = cost.eval(feed_dict = {X: X_train, y: y_train, Z: dprio.reshape(4,366)})
        pred = sess.run(yhat, feed_dict={X:X_test,Z:dprio.reshape(4,366)})
        print("Epoch: {}, loss: {}".format(i, loss))
        print(pred)
    #saver.save(sess, "C:/Users/dkoch/Documents/tensorflow/tmp/model.ckpt")
    builder.add_meta_graph_and_variables(sess,
                                       [tf.saved_model.tag_constants.TRAINING],
                                       signature_def_map=None,
                                       assets_collection=None)
builder.save()

Epoch: 0, loss: 2.133460521697998
[[0.01124158 0.01123397 0.01122637 ... 0.01009146 0.01009438 0.01009735]
 [0.00978915 0.00978285 0.00977655 ... 0.00861561 0.00861672 0.00861786]
 [0.60523593 0.60520667 0.605176   ... 0.5085119  0.5080344  0.5075559 ]
 ...
 [0.34032166 0.34057903 0.34083623 ... 0.37999076 0.3798386  0.37968445]
 [0.06221456 0.06216052 0.06210632 ... 0.0348041  0.03472061 0.03463714]
 [0.05722251 0.05717514 0.05712759 ... 0.03218559 0.03210766 0.03202974]]
Epoch: 1, loss: 2.132349967956543
[[0.01115726 0.01114963 0.01114201 ... 0.00998796 0.00999077 0.00999362]
 [0.00972714 0.00972081 0.00971448 ... 0.00853735 0.00853838 0.00853945]
 [0.59654963 0.59651554 0.59648013 ... 0.4994544  0.49898008 0.49850485]
 ...
 [0.3350843  0.33533376 0.33558306 ... 0.37218577 0.37203014 0.37187266]
 [0.06195864 0.06190434 0.06184985 ... 0.03456352 0.03448035 0.03439722]
 [0.05723175 0.05718381 0.05713569 ... 0.0320936  0.03201569 0.03193781]]
Epoch: 2, loss: 2.131371021270752
[[0.011100

 [0.07833245 0.07824925 0.0781658  ... 0.04066821 0.04056197 0.04045582]]
Epoch: 47, loss: 2.119368076324463
[[0.01426689 0.01425458 0.0142423  ... 0.01190753 0.01190844 0.01190938]
 [0.01288482 0.01287401 0.01286321 ... 0.01053732 0.01053654 0.01053579]
 [0.59416914 0.5939886  0.59380674 ... 0.45020616 0.4496417  0.4490767 ]
 ...
 [0.34538138 0.34554482 0.34570795 ... 0.34261942 0.34235352 0.34208602]
 [0.07954917 0.079464   0.07937862 ... 0.04120958 0.04110183 0.04099417]
 [0.0788651  0.07878102 0.07869674 ... 0.0408849  0.04077793 0.04067104]]
Epoch: 48, loss: 2.119164228439331
[[0.01437038 0.01435793 0.01434551 ... 0.01197756 0.01197841 0.01197931]
 [0.01298827 0.01297732 0.01296638 ... 0.01060724 0.01060641 0.01060562]
 [0.59491843 0.59473467 0.5945496  ... 0.44985014 0.44928336 0.448716  ]
 ...
 [0.34612277 0.34628472 0.34644634 ... 0.34259206 0.34232384 0.3420539 ]
 [0.08005811 0.07997212 0.07988592 ... 0.04141407 0.04130561 0.04119722]
 [0.07940169 0.07931674 0.0792316  ... 0.0

Epoch: 95, loss: 2.106618881225586
[[0.02133395 0.02131211 0.02129031 ... 0.01670545 0.01670356 0.01670173]
 [0.02007212 0.02005176 0.02003142 ... 0.01537751 0.01537362 0.01536979]
 [0.6348511  0.6345028  0.6341531  ... 0.43535048 0.43467143 0.43399218]
 ...
 [0.3837439  0.38382596 0.38390744 ... 0.3420081  0.34162813 0.34124652]
 [0.11308592 0.11294678 0.11280737 ... 0.05482369 0.054669   0.05451449]
 [0.11302934 0.1128903  0.11275098 ... 0.05479895 0.05464434 0.05448988]]
Epoch: 96, loss: 2.106250047683716
[[0.02154869 0.02152656 0.02150447 ... 0.01685179 0.01684982 0.01684792]
 [0.02029498 0.02027431 0.02025368 ... 0.01552732 0.01552333 0.0155194 ]
 [0.6358739  0.63552177 0.63516825 ... 0.43512592 0.43444446 0.43376264]
 ...
 [0.38464215 0.3847223  0.3848019  ... 0.3420399  0.34165743 0.34127343]
 [0.11407832 0.11393757 0.11379659 ... 0.05523106 0.055075   0.0549191 ]
 [0.1140257  0.11388507 0.11374416 ... 0.05520809 0.05505208 0.05489625]]
Epoch: 97, loss: 2.10587477684021
[[0.0217

NameError: name 'builder' is not defined

In [ ]:
check_sum=[]
for i in range(len(pred)):
    check_sum.append(np.sum(pred[i]))

In [ ]:
check_sum

In [ ]:
Data.loc[Data['calendar_date'].apply(lambda x: '2019' in x),:].groupby(['calendar_date','direction'])['pax_total'].sum()

In [ ]:
plt.plot(pred[0][range(365,-1,-1)])

In [ ]:
plt.plot(pred[10][range(365,-1,-1)])

In [ ]:
plt.plot(pred[8][range(365,-1,-1)])

In [ ]:
np.diff(pred[35],2)

In [ ]:
Data['calendar_date'].unique()

In [ ]:
export_dir="C:/Users/dkoch/Documents/tensorflow/tmp/model.ckpt"
with tf.Session(graph=tf.Graph()) as sess:
    tf.saved_model.loader.load(sess, [tf.saved_model.tag_constants.TRAINING], export_dir)